In [1]:
import asd

ModuleNotFoundError: No module named 'asd'

In [1]:
import json
import feedparser
from IPython.display import JSON
import requests
import datetime
feeds = [
    "https://anchor.fm/s/df98870/podcast/rss",
    "https://feeds.buzzsprout.com/1154870.rss",
    "http://feeds.soundcloud.com/users/soundcloud:users:294272020/sounds.rss",
    "https://feeds.acast.com/public/shows/5eded114811f0313e89ddaeb",
    "https://regenerativeagriculturepodcast.com/rss",
    # "https://regenerativeskills.com/feed/podcast/",
    "https://feeds.backtracks.fm/feeds/tenacious/agtech-so-what/feed.xml?1667419322",
    "https://feeds.libsyn.com/301781/rss",
    "https://anchor.fm/s/10c2c51c/podcast/rss",
    "https://feeds.buzzsprout.com/1857893.rss",
    "https://feeds.buzzsprout.com/1303834.rss",
    "https://feeds.transistor.fm/regenerative-by-design"
]

feed = feeds[1]

In [2]:
podcast = feedparser.parse(feed)
feed_meta = podcast["feed"]

In [3]:
entry = podcast["entries"][1]

In [101]:
JSON(entry)

<IPython.core.display.JSON object>

In [20]:
JSON(entry)

<IPython.core.display.JSON object>

In [37]:
def get_chapters(entry):
    chapters = requests.get(entry["podcast_chapters"]["url"]).json()["chapters"]
    for chapter in chapters:
        chapter["start"] = str(datetime.timedelta(seconds=chapter.pop("startTime")))

    return chapters
    
def get_audio(entry):
    audio = list(filter(lambda x: x if x["type"] == "audio/mpeg" else False, entry["links"]))[0]
    
    return [
        {
          "url": audio["href"],
          "size": audio["length"],
          "title": "MP3 Audio (mp3)",
          "mimeType": audio["type"]
        }
    ]

[{'title': 'Why are you doing what you are doing? Why Soil?',
  'start': '0:02:19.610000'},
 {'title': 'Role of culture and intergenerational learning with animals',
  'start': '0:18:21.610000'},
 {'title': 'What does that mean for humans?', 'start': '0:26:43.610000'},
 {'title': 'Why we are not eating enough healthy whole some foods',
  'start': '0:33:33.610000'},
 {'title': 'What can we learn from how plants manage their economies?',
  'start': '0:43:31.610000'},
 {'title': 'What we lost by selecting for yield', 'start': '0:48:26.610000'},
 {'title': 'What would you do if you were in charge of a 1B investment portfolio',
  'start': '0:57:33.610000'},
 {'title': 'If you could wave a magic wand and change one thing in the agriculture industry',
  'start': '1:08:04.610000'}]

In [44]:
JSON(entry)

<IPython.core.display.JSON object>

In [67]:
import pickle
with open("transcriptions/Investing in Regenerative Agriculture and Food/169-cath-tayleur-on-why-measuring-biodiversity-is-one-of-the-key-levers-for-regenerative-agriculture.pickle", "rb") as f:
    transcript_raw = pickle.load(f)

3375680.0

In [75]:
from podcast_transcription import batch

def get_podlove_transcripts(transcript, batch_size=3):
    podlove_transcripts = []
    for seq in batch(transcript["segments"], batch_size):
        start = seq[0]["start"]
        end = seq[-1]["end"]
        text = " ".join([x["text"] for x in seq])
        
        podlove_transcripts.append({
              "start": str(datetime.timedelta(seconds=start)),
              "start_ms": start*1000,
              "end": str(datetime.timedelta(seconds=end)),
              "end_ms": end*1000,
              "text": text
            })
    return podlove_transcripts

In [92]:
def get_podlove_config(feed_meta):
    return {
        "version": 5,
        "subscribe-button": {
        "feed": [x for x in feed_meta["links"] if "rss" in x["type"] or "atom" in x["type"]][0]["href"],
        "clients": [
          {
            "id": "apple-podcasts",
          },
          {
            "id": "antenna-pod"
          },
          {
            "id": "beyond-pod"
          },
          {
            "id": "castro"
          },
          {
            "id": "clementine"
          },
          {
            "id": "downcast"
          },
          {
            "id": "google-podcasts",
          },
          {
            "id": "gpodder"
          },
          {
            "id": "itunes"
          },
          {
            "id": "i-catcher"
          },
          {
            "id": "instacast"
          },
          {
            "id": "overcast"
          },
          {
            "id": "player-fm"
          },
          {
            "id": "pocket-casts"
          },
          {
            "id": "pocket-casts",
          },
          {
            "id": "pod-grasp"
          },
          {
            "id": "podcast-addict"
          },
          {
            "id": "podcast-republic"
          },
          {
            "id": "podcat"
          },
          {
            "id": "podscout"
          },
          {
            "id": "rss-radio"
          },
          {
            "id": "rss"
          }
        ]
        },

        "playlist": "playlist.json",

        "share": {
        "channels": [
          "facebook",
          "twitter",
          "whats-app",
          "linkedin",
          "pinterest",
          "xing",
          "mail",
          "link"
        ],
        "sharePlaytime": true
        },

        "features": {
        "persistTab": true,
        "persistPlaystate": true
        }
    }
